<a href="https://colab.research.google.com/github/BYU-Hydroinformatics/baseflow-notebooks/blob/main/baseflow_multi_station.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***In this notebook, we will demonstrate how to use the baseflow package to perform baseflow separation on streamflow data obtained from multiple stations.***

Workflow:
 * Data Acquisition: Fetch streamflow data from the USGS API or upload your own data files.
 * Data Preparation: Prepare the data for analysis, including converting timestamps and handling missing values.
 * Baseflow Separation: Apply various baseflow separation methods provided by the baseflow package to each station's data.
 * Result Analysis: Visualize the separated baseflow components and calculate performance metrics for each station.

***By the end of this notebook, you will have a comprehensive understanding of how to perform baseflow separation for multiple stations using the baseflow package and analyze the results effectively.***

# Install baseflow package from github

In [ ]:
!pip install git+https://github.com/BYU-Hydroinformatics/baseflow.git

  Cloning https://github.com/BYU-Hydroinformatics/baseflow.git to /tmp/pip-req-build-9zl6zhwu
  Running command git clone --filter=blob:none --quiet https://github.com/BYU-Hydroinformatics/baseflow.git /tmp/pip-req-build-9zl6zhwu
  Resolved https://github.com/BYU-Hydroinformatics/baseflow.git to commit 50a60ee97f9132f06a30b454ab0d8c5eb4dab97d
  Preparing metadata (setup.py) ... done
  Created wheel for baseflow: filename=baseflow-0.0.9-py3-none-any.whl size=101938 sha256=89673cd03703dc5af9307430df6986786f91c12b7195fdc8686f3aba3b675008
  Stored in directory: /tmp/pip-ephem-wheel-cache-_8w63oda/wheels/f4/6d/fe/c937a4db1017b5070652dfe2b4841589a4aca8cff895ae4abe
Successfully built baseflow


# Load necessary packages

In [ ]:
import baseflow
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import urllib.parse
import urllib.request
import os
import numpy as np
from numba import njit, prange

# Download flow data from USGS

This part of the code essentially automates the process of fetching streamflow data for a specific station and time period from the USGS website. It prepares the data for further processing and analysis in the subsequent steps of your notebook.

This part includes:

*   Specify Parameters: You define the station_number, start_date, and end_date to customize the data retrieval.

*   Construct URL: The code constructs the URL for the USGS API request by concatenating different sections. Each section contains specific parameters and filters for the data you want.

*   Retrieve Data: The urllib.request.urlopen(link) function sends a request to the constructed URL and retrieves the data from the USGS website.

*   Decode and Split: The retrieved data is decoded from bytes to a string using .decode(), and then split into lines using .split('\n').

*   Extract Station Name: The code iterates through the lines and extracts the station name from the line starting with "# USGS".

### Specify station number and dates

In this section, you need to type the single streamflow station number, start date and end date of the time period.

You can find stations on USGS website: https://dashboard.waterdata.usgs.gov/app/nwd/en/

In [ ]:
station_number = '01636500'
start_date = '2000-01-01' # (YYYY-MM-DD)
end_date = '2022-12-31' # (YYYY-MM-DD)
folder = os.getcwd()

### Scrape USGS Website

Run this cell and it will return this link corresponding to your input. Open the link and check the site number, name, date, etc.

In [ ]:
section1 = 'https://nwis.waterdata.usgs.gov/nwis/dv?referred_module=sw&search_site_no='
section2 = '&search_site_no_match_type=exact&site_tp_cd=OC&site_tp_cd=OC-CO&site_tp_cd=ES&site_tp_cd='\
'LK&site_tp_cd=ST&site_tp_cd=ST-CA&site_tp_cd=ST-DCH&site_tp_cd=ST-TS&index_pmcode_00060=1&group_key='\
'NONE&sitefile_output_format=html_table&column_name=agency_cd&column_name=site_no&column_name=station_nm&range_selection=date_range&begin_date='
section3 = '&end_date='
section4 = '&format=rdb&date_format=YYYY-MM-DD&rdb_compression=value&list_of_search_criteria=search_site_no%2Csite_tp_cd%2Crealtime_parameter_selection'

link = (section1 + station_number + section2 + start_date + section3 + end_date + section4)
print("Click here to see the generated USGS link: \n",link)

USGS_page = urllib.request.urlopen(link)
downloaded_data = USGS_page.read()
str_data = downloaded_data.decode()
f_str_data = str_data.split('\n')
station_name = ''

for line in range(len(f_str_data)):
    if f_str_data[line].startswith("#    USGS"):
        station_name=f_str_data[line][3:]
print(station_name)


Click here to see the generated USGS link: 
 https://nwis.waterdata.usgs.gov/nwis/dv?referred_module=sw&search_site_no=01636500&search_site_no_match_type=exact&site_tp_cd=OC&site_tp_cd=OC-CO&site_tp_cd=ES&site_tp_cd=LK&site_tp_cd=ST&site_tp_cd=ST-CA&site_tp_cd=ST-DCH&site_tp_cd=ST-TS&index_pmcode_00060=1&group_key=NONE&sitefile_output_format=html_table&column_name=agency_cd&column_name=site_no&column_name=station_nm&range_selection=date_range&begin_date=2000-01-01&end_date=2022-12-31&format=rdb&date_format=YYYY-MM-DD&rdb_compression=value&list_of_search_criteria=search_site_no%2Csite_tp_cd%2Crealtime_parameter_selection
  USGS 01636500 SHENANDOAH RIVER AT MILLVILLE, WV


### Format data
As you have got the link to the station, this block can make the information on the page into a DataFrame with date and streamflow.

In [ ]:
date_flow = ''

for line in range(len(f_str_data)):
    if f_str_data[line].startswith("USGS"):
        data = f_str_data[line][14:]
        columns = data.split('\t')
        rows = ','.join([columns[0],(columns[1])])
        date_flow += rows + '\n'
date_flow = date_flow.encode()

with open(folder+'/USGS_Data_for_' + station_number  + '.txt', 'wb') as text:
        text.write(date_flow)

filename = folder+'/USGS_Data_for_' + station_number  + '.txt'
columns = ['Date','Discharge (cfs)']
df = pd.read_csv(filename,header=None,names=columns,parse_dates=[0])
df=df.set_index(['Date'])
df['Discharge (cfs)']=pd.to_numeric(df['Discharge (cfs)'], errors='coerce')
df.tail()

,Discharge (cfs)
Date,
2022-12-27,4700
2022-12-28,3790
2022-12-29,3230
2022-12-30,2880
2022-12-31,2630


# Helper fuctions

We created a function called multi_station and the main purpose of this is to separate the baseflow for multiple stations using different methods and optionally calculate performance metrics like Baseflow Index (BFI) and Kling-Gupta Efficiency (KGE).

In [ ]:
def multi_station(df, df_sta=None, method='all', return_bfi=False, return_kge=False):
    # baseflow separation worker for single station
    def sep_work(s):
        try:
            # read area, longitude, latitude from df_sta
            area, ice = None, None
            to_num = lambda col: (pd.to_numeric(df_sta.loc[s, col], errors='coerce')
                                  if (df_sta is not None) and (col in df_sta.columns) else np.nan)
            if np.isfinite(to_num('area')):
                area = to_num('area')
            if np.isfinite(to_num('lon')):
                c, r = geo2imagexy(to_num('lon'), to_num('lat'))
                # ice = ~thawed[:, r, c]
                ice = ([11, 1], [3, 31]) if ice.all() else ice
            # separate baseflow for station S
            b, KGEs = single(df[s], ice=ice, area=area, method=method, return_kge=return_kge)
            # write into already created dataframe
            for m in method:
                dfs[m].loc[b.index, s] = b[m]
            if return_bfi:
                df_bfi.loc[s] = b.sum() / df.loc[b.index, s].abs().sum()
            if return_kge:
                df_kge.loc[s] = KGEs
        except BaseException:
            pass

    # convert index to datetime
    if not isinstance(df.index, pd.DatetimeIndex):
        df.index = pd.to_datetime(df.index)

    # # thawed months from https://doi.org/10.5194/essd-9-133-2017
    # with np.load(Path(__file__).parent / 'thawed.npz') as f:
    #     thawed = f['thawed']

    # create df to store baseflow
    method = format_method(method)
    dfs = {m: pd.DataFrame(np.nan, index=df.index, columns=df.columns, dtype=float)
           for m in method}

    # create df to store BFI and KGE
    if return_bfi:
        df_bfi = pd.DataFrame(np.nan, index=df.columns, columns=method, dtype=float)
    if return_kge:
        df_kge = pd.DataFrame(np.nan, index=df.columns, columns=method, dtype=float)

    # run separation for each column
    for s in tqdm(df.columns, total=df.shape[1]):
        sep_work(s)

    # return result
    if return_bfi and return_kge:
        return dfs, df_bfi, df_kge
    if return_bfi and not return_kge:
        return dfs, df_bfi
    if not return_bfi and return_kge:
        return dfs, df_kge
    return dfs

# Calculate baseflow at multiple stations
In this example, we use the data from the USGS API as input. You can also upload your own file to the side bar by clicking the 'Files' button.

In [ ]:
# Example station IDs and date range
station_ids = ['station_1', 'station_2', 'station_3']
start_date = '2023-01-01'
end_date = '2023-12-31'

# Fetch the data
df = fetch_usgs_data(station_ids, start_date, end_date)


Prepare Station Metadata (Optional)

If you have additional metadata for the stations, such as area and geographical coordinates, you can prepare a DataFrame for it. This step is optional.

In [ ]:
df_sta = pd.DataFrame({
    'station_id': station_ids,
    'area': [100, 150, 200],  # Example areas in km^2
    'lon': [-100.0, -101.0, -102.0],  # Example longitudes
    'lat': [40.0, 41.0, 42.0]  # Example latitudes
}).set_index('station_id')


Now, we will use the multi_stations function to perform baseflow separation on the fetched data.

In [ ]:
# Define the methods to use for baseflow separation
methods = ['lh', 'fixed', 'furey']  # Example methods

# Perform baseflow separation
dfs, df_bfi, df_kge = multi_station(df, df_sta=df_sta, method=methods, return_bfi=True, return_kge=True)

# Display the results
print("Baseflow Separation Results:")
for method, result_df in dfs.items():
    print(f"\nMethod: {method}")
    print(result_df.head())

print("\nBaseflow Index (BFI):")
print(df_bfi)

print("\nKling-Gupta Efficiency (KGE):")
print(df_kge)
`



You can visualize the separated baseflow components using popular Python libraries such as Matplotlib.

In [ ]:
import matplotlib.pyplot as plt

# Plot the original streamflow and separated baseflow for one station and one method
station_id = 'station_1'
method = 'lh'

plt.figure(figsize=(12, 6))
plt.plot(df.index, df[station_id], label='Original Streamflow')
plt.plot(dfs[method].index, dfs[method][station_id], label='Separated Baseflow')
plt.xlabel('Date')
plt.ylabel('Streamflow (cfs)')
plt.title(f'Baseflow Separation for {station_id} using {method} Method')
plt.legend()
plt.show()
